<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/filtra_deter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pacotes
import pathlib
from osgeo import ogr, gdal, gdalconst
from skimage.morphology import disk, dilation, erosion
import os
import numpy as np
import shutil
from google.colab import drive
import cv2
from datetime import date, timedelta
import pandas as pd

In [ ]:
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [ ]:
def converte_deter_tif(path_to_deter, output_path, patch_name, x_size, y_size, extent_tuple, d_begin, d_end):
  # Abrindo arquivo
  v_deter = ogr.Open(path_to_deter)
  l_deter = v_deter.GetLayer()
  # Definindo localização
  x_res = (extent_tuple[2]-extent_tuple[0])/x_size
  y_res = (extent_tuple[3]-extent_tuple[1])/y_size
  geo_transform = [extent_tuple[0], x_res, 0, extent_tuple[1], 0, y_res]
  # Nome de saída
  output = os.path.join(output_path, f'{patch_name}.tif')
  # Filtro de datas sendo aplicado
  where_ref = f'(cast(\"VIEW_DATE\" as character) <\'{d_end}\') AND ( cast(\"VIEW_DATE\" as character) >= \'{d_begin}\' )'
  # Objeto que manipula os dados
  target_ds = gdal.GetDriverByName('GTiff').Create(output, x_size, y_size, 1, gdal.GDT_Byte)
  target_ds.SetGeoTransform(geo_transform)
  l_deter.ResetReading()
  l_deter.SetAttributeFilter(where_ref)
  print (l_deter.GetFeatureCount())
  # Salvando
  gdal.RasterizeLayer(target_ds, [1], l_deter, burn_values=[1])
  # Limpando
  target_ds.FlushCache()
  del target_ds
  #
  return 1

In [ ]:
# Parâmetros fixos
output_path='/content/drive/My Drive/projeto_DL/deter_tratado/'
deterFile="/content/drive/My Drive/projeto_DL/dados_brutos/deter_public.shp"
x_size = 256*10
y_size = 256*10
extent= [-59.9206710380000018, -7.3125828444699836, -59.1668605229999969, -6.7566813625300162] 

In [ ]:
# Rodando em loop - versao mensal

# Construindo intervalos de datas
anos_rodar = ['2015','2016','2017','2018','2019','2020','2021', '2022']
l_d_begin = []
for i in anos_rodar:
  for j in ['01','02','03','04','05','06','07','08','09','10','11','12']:
    l_d_begin.append(i+"/"+j+"/"+"01")

l_d_end = l_d_begin.copy()
del l_d_end[0]
l_d_end.append('2022/01/01')

# Loop
for i in range(len(l_d_begin)):
  patch_name=f"{i}"
  d_begin = l_d_begin[i]
  d_end = l_d_end[i]
  converte_deter_tif(deterFile, output_path + 'mensal/', patch_name, x_size, y_size, extent, d_begin, d_end)
  print((i+1)/len(l_d_begin))

0
0.010416666666666666
0
0.020833333333333332
0
0.03125
0
0.041666666666666664
0
0.052083333333333336
0
0.0625
0
0.07291666666666667
0
0.08333333333333333
0
0.09375
0
0.10416666666666667
0
0.11458333333333333
0
0.125
0
0.13541666666666666
0
0.14583333333333334
0
0.15625
0
0.16666666666666666
0
0.17708333333333334
0
0.1875
0
0.19791666666666666
8300
0.20833333333333334
6007
0.21875
7537
0.22916666666666666
2920
0.23958333333333334
67
0.25
246
0.2604166666666667
325
0.2708333333333333
243
0.28125
427
0.2916666666666667
1238
0.3020833333333333
2417
0.3125
2018
0.3229166666666667
1807
0.3333333333333333
6581
0.34375
4869
0.3541666666666667
3111
0.3645833333333333
2341
0.375
1480
0.3854166666666667
1287
0.3958333333333333
2272
0.40625
2438
0.4166666666666667
2611
0.4270833333333333
3784
0.4375
4737
0.4479166666666667
5140
0.4583333333333333
6334
0.46875
3555
0.4791666666666667
2057
0.4895833333333333
365
0.5
586
0.5104166666666666
672
0.5208333333333334
1494
0.53125
1605
0.5416666666666666


In [ ]:
pd.DataFrame({'begin':l_d_begin, 'end':l_d_end, 'patch_name':np.arange(len(l_d_end))}).to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_mes.csv')

In [ ]:
# Rodando em loop - versao semanal

# Pegando todos os domingos do ano
# https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python
def all_wdays(year, wday = 6):
   d = date(year, 1, 1)                    # January 1st
   d += timedelta(days = wday - d.weekday())  # 0 : Monday ... 6 : Sunday
   if d.year != year:
    d += timedelta(days = 7)
   while d.year == year:
      yield d
      d += timedelta(days = 7)

# Montando datas
# Começo da semana
anos_rodar = [2015,2016,2017,2018,2019,2020,2021,2022]
l_d_begin = []
for i in anos_rodar:
  for d in all_wdays(i):
    l_d_begin.append(d)

# Fim da semana
l_d_end = []
for i in l_d_begin:
  l_d_end.append(i+timedelta(days = 6))

# Loop
for i in range(len(l_d_begin)):
  patch_name=f"{i}"
  d_begin = l_d_begin[i].strftime("%Y/%m/%d")
  d_end = l_d_end[i].strftime("%Y/%m/%d")
  converte_deter_tif(deterFile, output_path + 'semanal/', patch_name, x_size, y_size, extent, d_begin, d_end)
  print((i+1)/len(l_d_begin))

0
0.002398081534772182
0
0.004796163069544364
0
0.007194244604316547
0
0.009592326139088728
0
0.011990407673860911
0
0.014388489208633094
0
0.016786570743405275
0
0.019184652278177457
0
0.02158273381294964
0
0.023980815347721823
0
0.026378896882494004
0
0.02877697841726619
0
0.03117505995203837
0
0.03357314148681055
0
0.03597122302158273
0
0.03836930455635491
0
0.0407673860911271
0
0.04316546762589928
0
0.045563549160671464
0
0.047961630695443645
0
0.050359712230215826
0
0.05275779376498801
0
0.05515587529976019
0
0.05755395683453238
0
0.05995203836930456
0
0.06235011990407674
0
0.06474820143884892
0
0.0671462829736211
0
0.06954436450839328
0
0.07194244604316546
0
0.07434052757793765
0
0.07673860911270983
0
0.07913669064748201
0
0.0815347721822542
0
0.08393285371702638
0
0.08633093525179857
0
0.08872901678657075
0
0.09112709832134293
0
0.09352517985611511
0
0.09592326139088729
0
0.09832134292565947
0
0.10071942446043165
0
0.10311750599520383
0
0.10551558752997602
0
0.1079136690647482
0

In [ ]:
pd.DataFrame({'begin':l_d_begin, 'end':l_d_end, 'patch_name':np.arange(len(l_d_end))}).to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_semana.csv')